# Example dot tuning

In [ ]:
from qcodes import Station
import nanotune as nt
from nanotune.tuningstages.settings import DataSettings, SetpointSettings, Classifiers
from nanotune.tests.functional_tests.sim_tuner import SimDotTuner
from nanotune.drivers.mock_dac import MockDAC, MockDACChannel
from nanotune.tests.mock_classifier import MockClassifer
from sim.qcodes_mocks import MockDoubleQuantumDotInstrument
from load_sim_scenarios import sim_scenario_dottuning
from load_sim_device import load_sim_device

# Load instruments and simulator to play back data
We use a MockDAC and a simulator to replace a real setup.
The simulator takes previously measured data and plays it back. It needs to be initialized and added to the station.

In [ ]:
station = Station()
dac = MockDAC('dac', MockDACChannel)
station.add_component(dac)

qd_mock_instrument = MockDoubleQuantumDotInstrument('qd_mock_instrument')
station.add_component(qd_mock_instrument, name="qd_mock_instrument")

Now we load the device and overwrite the channel's voltage parameters with the simulator's pin voltages.
Then the simulation scenario is loaded, i.e. we tell the simulator which data to play back and update the device's normalization constants. In a real measurement, the normalization constants need to be measured. Unfortunately this is currently not supported by the simulator.

In [ ]:
device = load_sim_device(station, "./chip.yaml")
sim_sceanrio = sim_scenario_dottuning(station)
device.normalization_constants

# Initialize tuner
We use the SimDotTuner here, which is a subclass of the DotTuner with the only difference that before each measurement, the data simulated is advanced to replay the next dataset.

In [ ]:
settings = {
    "name": "test_tuner",
    "data_settings": DataSettings(
        db_name="dot_tuning_example.db",
        db_folder='.',
        segment_db_name="dot_segments_temp.db",
        segment_db_folder='.',
    ),
    "classifiers": Classifiers(
        pinchoff=MockClassifer(category="pinchoff"),
        singledot=MockClassifer(category="singledot"),
        doubledot=MockClassifer(category="doubledot"),
        dotregime=MockClassifer(category="dotregime")),
    "setpoint_settings": SetpointSettings(
        voltage_precision=0.001,
        ranges_to_sweep=[(-1, 0)],
        safety_voltage_ranges=[(-2, 0)],
    ),
}

In [ ]:
dottuner = SimDotTuner(
    **settings, 
    sim_scenario=sim_sceanrio,
)

In [ ]:
dottuner.tune_dot_regime(device)

The tuning result is saved in the tuner's tuning history.

In [ ]:
dottuner.tuning_history.results[device.name].tuningresults.keys()

The valid ranges found during tuning can be accessed via the current valid ranges attribute.

In [ ]:
device.current_valid_ranges()